In [119]:
!ls ../shared_data_read_only/

 2021_CBP_NAICS_ECON_DATA.csv
'DFG - Data Dictionary .xlsx'
 District_Value_Added_Ranking.xlsx
 District_and_School_Performance_Index_Ranking.xlsx
 Documentation
 Greenspace.csv
 HOME_IncomeLmts_Natl_2023.xlsx
 HUD_Rent_Data.csv
 NCES_School_Lookup.csv
 Walkability.csv
 geography_mappings


In [120]:
import pandas as pd
import numpy as np

In [121]:
# Load the Excel file into a DataFrame
school_ranking = pd.read_excel('../shared_data_read_only/District_and_School_Performance_Index_Ranking.xlsx', sheet_name='LEA PI Rankings')

#print(school_ranking)
school_ranking = school_ranking.replace("NC", 'NaN')
school_ranking['2023 PI for Ranking'] = school_ranking['2023 PI for Ranking'].astype(float)

#print(school_ranking)
skl_df = school_ranking.groupby('ODE Designated County', dropna = True)['2023 PI for Ranking'].agg(['min', 'max']).reset_index()
skl_df["DIFFSklPI"] = skl_df["max"] - skl_df["min"]
skl_df = skl_df.rename(columns={'2023 PI for Ranking': 'skl_PI_ranking'})
print(skl_df)
#print(sorted_skl)

# Load the Excel file into a DataFrame
build_ranking = pd.read_excel('../shared_data_read_only/District_and_School_Performance_Index_Ranking.xlsx', sheet_name='Building PI Rankings')
#print(build_ranking)
build_ranking = build_ranking.replace("NC", 'NaN')
build_ranking['2023 PI for Ranking'] = build_ranking['2023 PI for Ranking'].astype(float)

#print(build_ranking)
build_df = build_ranking.groupby('ODE Designated County', dropna = True)['2023 PI for Ranking'].agg(['min', 'max']).reset_index()
build_df["DIFFBuildPI"] = build_df["max"] - build_df["min"]
build_df = build_df.rename(columns={'2023 PI for Ranking': 'building_PI_ranking'})

merged_df = pd.merge(skl_df, build_df, on='ODE Designated County', how='outer')
new_df = merged_df[['ODE Designated County', 'DIFFSklPI', 'DIFFBuildPI']]
print(new_df)

df = pd.read_csv("data/places.txt", sep= "|")
df['COUNTYNAME'] = df['COUNTYNAME'].str.replace(' County', '')


merge = pd.merge(df, new_df, left_on='COUNTYNAME', right_on='ODE Designated County', how='inner')
print(merge)

   ODE Designated County     min      max  DIFFSklPI
0                  Adams  80.378   82.082      1.704
1                  Allen  61.542   98.058     36.516
2                Ashland  84.508   92.666      8.158
3              Ashtabula  66.330   84.823     18.493
4                 Athens  67.254   86.584     19.330
..                   ...     ...      ...        ...
83            Washington  75.083   90.608     15.525
84                 Wayne  82.132   98.632     16.500
85              Williams  82.186   92.146      9.960
86                  Wood  79.677  102.656     22.979
87               Wyandot  88.298   96.528      8.230

[88 rows x 4 columns]
   ODE Designated County  DIFFSklPI  DIFFBuildPI
0                  Adams      1.704       21.210
1                  Allen     36.516       57.529
2                Ashland      8.158       37.423
3              Ashtabula     18.493       36.952
4                 Athens     19.330       30.212
..                   ...        ...          ..

In [122]:
# df = pd.read_csv("data/zip.txt", sep= "\t", header = None, names = ['zip_code','???', 'City', 'County', 'State', 'area_num'])
# df['County'] = df['County'].str.replace(' County', '')
# df['zip_code'] = df['zip_code'].astype(int)
    

merge['Key'] = merge['PLACENAME'].str.extract(r'^(.*?)\s\w+$', expand=False)

merge[merge['Key'] == 'Columbus']
# df['Key'] = df['City']

# merged_df = df.merge(merge, on='Key', how='inner')
# print(merged_df)

,STATE,STATEFP,COUNTYFP,COUNTYNAME,PLACEFP,PLACENS,PLACENAME,TYPE,CLASSFP,FUNCSTAT,ODE Designated County,DIFFSklPI,DIFFBuildPI,Key
319,OH,39,41,Delaware,18000,1086101,Columbus city,INCORPORATED PLACE,C6,A,Delaware,16.563,35.186,Columbus
351,OH,39,45,Fairfield,18000,1086101,Columbus city,INCORPORATED PLACE,C6,A,Fairfield,13.769,40.710,Columbus
379,OH,39,49,Franklin,18000,1086101,Columbus city,INCORPORATED PLACE,C6,A,Franklin,69.463,82.500,Columbus


In [123]:
# rent_df = pd.read_csv("rent.csv", sep = '\t')

# total_df = pd.merge(merged_df, rent_df, left_on='zip_code', right_on='ZIP\nCode', how='inner')
# print(total_df)

In [124]:
income_df = pd.read_csv('income.csv', sep = '\t')

total_df = pd.merge(merge, income_df, left_on='COUNTYFP', right_on='FIPS County Code', how='inner')
print(total_df[total_df['Key'] == 'Columbus'])

    STATE  STATEFP  COUNTYFP COUNTYNAME  PLACEFP  PLACENS      PLACENAME  \
319    OH       39        41   Delaware    18000  1086101  Columbus city   
351    OH       39        45  Fairfield    18000  1086101  Columbus city   
379    OH       39        49   Franklin    18000  1086101  Columbus city   

                   TYPE CLASSFP FUNCSTAT  ... FIPS State Code  \
319  INCORPORATED PLACE      C6        A  ...              39   
351  INCORPORATED PLACE      C6        A  ...              39   
379  INCORPORATED PLACE      C6        A  ...              39   

     FIPS County Code  medinc_23p1 medinc_23p2  medinc_23p3  medinc_23p4  \
319                41        69500       79400        89300        99200   
351                45        69500       79400        89300        99200   
379                49        69500       79400        89300        99200   

     medinc_23p5  medinc_23p6  medinc_23p7  medinc_23p8  
319       107200       115100       123100       131000  
351       107

In [113]:
schools = pd.read_csv('../shared_data_read_only/NCES_School_Lookup.csv', skiprows=6, skipfooter=7)

schools['County Name [Public School] 2018-19'] = schools['County Name [Public School] 2018-19'].str.replace(' County', '')
schools['Free Lunch Eligible [Public School] 2018-19'] = schools['Free Lunch Eligible [Public School] 2018-19'].str.replace('–', '0')
schools['Reduced-price Lunch Eligible Students [Public School] 2018-19'] = schools['Reduced-price Lunch Eligible Students [Public School] 2018-19'].str.replace('–', '0')
schools['Free and Reduced Lunch Students [Public School] 2018-19'] = schools['Free and Reduced Lunch Students [Public School] 2018-19'].str.replace('–', '0')

schools.rename(columns={'State Name [Public School] Latest available year': 'State Name'}, inplace=True)
schools.rename(columns={'Agency Name [Public School] 2018-19': 'Agency Name'}, inplace=True)
schools.rename(columns={'County Name [Public School] 2018-19': 'County Name'}, inplace=True)
schools.rename(columns={'Location ZIP [Public School] 2018-19': 'Location ZIP'}, inplace=True)
schools.rename(columns={'Location City [Public School] 2018-19': 'Location City'}, inplace=True)
schools.rename(columns={'Total Students All Grades (Excludes AE) [Public School] 2018-19': 'Total Students'}, inplace=True)
schools.rename(columns={'Free Lunch Eligible [Public School] 2018-19': 'Free Lunch Eligible'}, inplace=True)
schools.rename(columns={'Pupil/Teacher Ratio [Public School] 2018-19': 'Pupil/Teacher ratio'}, inplace=True)
schools.rename(columns={'Reduced-price Lunch Eligible Students [Public School] 2018-19': 'Reduced-price lunch'}, inplace=True)
schools.rename(columns={'Free and Reduced Lunch Students [Public School] 2018-19': 'Free and reduced lunch'}, inplace=True)
schools.rename(columns={'Full-Time Equivalent (FTE) Teachers [Public School] 2018-19': 'Full-Time Teachers'}, inplace=True)
schools.rename(columns={'Full-Time Equivalent (FTE) Teachers [Public School] 2018-19': 'Full-Time Teachers'}, inplace=True)
schools.rename(columns={'Pupil/Teacher Ratio [Public School] 2018-19': 'Pupil/Teacher ratio'}, inplace=True)

schools['Total Students'] = schools['Total Students'].str.extract('(\d+)')
schools['Free and reduced lunch'] = schools['Free and reduced lunch'].str.extract('(\d+)')

schools = schools.dropna()

schools['Total Students'] = schools['Total Students'].astype(int)
schools['Free and reduced lunch'] = schools['Free and reduced lunch'].astype(int)

skls = schools.groupby('County Name', dropna = True)[['Total Students', 'Free and reduced lunch']].mean()
skls = skls.reset_index()
skls

total_df = pd.merge(total_df, skls, left_on='COUNTYNAME', right_on='County Name', how='outer')
print(total_df[total_df['Key'] == 'Columbus'])

    STATE  STATEFP  COUNTYFP COUNTYNAME  PLACEFP  PLACENS      PLACENAME  \
319    OH       39        41   Delaware    18000  1086101  Columbus city   
351    OH       39        45  Fairfield    18000  1086101  Columbus city   
379    OH       39        49   Franklin    18000  1086101  Columbus city   

                   TYPE CLASSFP FUNCSTAT  ... medinc_23p2  medinc_23p3  \
319  INCORPORATED PLACE      C6        A  ...       79400        89300   
351  INCORPORATED PLACE      C6        A  ...       79400        89300   
379  INCORPORATED PLACE      C6        A  ...       79400        89300   

     medinc_23p4 medinc_23p5  medinc_23p6  medinc_23p7  medinc_23p8  \
319        99200      107200       115100       123100       131000   
351        99200      107200       115100       123100       131000   
379        99200      107200       115100       123100       131000   

     County Name  Total Students  Free and reduced lunch  
319     Delaware      767.636364              108.2181

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [125]:
crime_df = pd.read_excel('ohio.xls', skiprows = 4)
crime_df['total_crime'] = crime_df.iloc[:, 2:].sum(axis=1)
crime_df['crime_rate'] = crime_df['total_crime'] / crime_df['Population'] * 100

new_df = crime_df[['City', 'crime_rate']]

#print(new_df[new_df['City'] == 'Columbus'])

# total_df['Key'] = total_df['Key'].str.lower().str.strip()

tot_df = pd.merge(total_df, new_df, right_on='City', left_on = 'Key', how='left')
tot_df[tot_df['Key'] == 'Columbus']
#tot_df

# df = pd.read_csv("walkability.csv", sep= "\t")

# df['CBSA_Name'] = df['CBSA_Name'].str.extract(r'^([^,]+)', expand=False)
# #print(df)

# total_df = pd.merge(total_df, df, left_on='Key', right_on='CBSA_Name', how='inner')
# print(total_df)

,STATE,STATEFP,COUNTYFP,COUNTYNAME,PLACEFP,PLACENS,PLACENAME,TYPE,CLASSFP,FUNCSTAT,...,medinc_23p1,medinc_23p2,medinc_23p3,medinc_23p4,medinc_23p5,medinc_23p6,medinc_23p7,medinc_23p8,City,crime_rate
319,OH,39,41,Delaware,18000,1086101,Columbus city,INCORPORATED PLACE,C6,A,...,69500,79400,89300,99200,107200,115100,123100,131000,NaN,NaN
351,OH,39,45,Fairfield,18000,1086101,Columbus city,INCORPORATED PLACE,C6,A,...,69500,79400,89300,99200,107200,115100,123100,131000,NaN,NaN
379,OH,39,49,Franklin,18000,1086101,Columbus city,INCORPORATED PLACE,C6,A,...,69500,79400,89300,99200,107200,115100,123100,131000,NaN,NaN


In [115]:
# total_df = total_df.dropna()
# total_df['SAFMR\n0BR'] = total_df['SAFMR\n0BR'].str.replace('[\$,]', '', regex=True).astype(int)
# total_df['SAFMR\n1BR'] = total_df['SAFMR\n1BR'].str.replace('[\$,]', '', regex=True).astype(int)
# total_df['SAFMR\n2BR'] = total_df['SAFMR\n2BR'].str.replace('[\$,]', '', regex=True).astype(int)
# total_df['SAFMR\n3BR'] = total_df['SAFMR\n3BR'].str.replace('[\$,]', '', regex=True).astype(int)
# total_df['SAFMR\n4BR'] = total_df['SAFMR\n4BR'].str.replace('[\$,]', '', regex=True).astype(int)

print(tot_df)

     STATE  STATEFP  COUNTYFP COUNTYNAME  PLACEFP  PLACENS  \
0       OH       39         1      Adams     5676  2628863   
1       OH       39         1      Adams    13834  2785006   
2       OH       39         1      Adams    47012  2399237   
3       OH       39         1      Adams    61420  2399644   
4       OH       39         1      Adams    68196  2399112   
...    ...      ...       ...        ...      ...      ...   
1351    OH       39       175    Wyandot    47992  2399262   
1352    OH       39       175    Wyandot    53942  2399447   
1353    OH       39       175    Wyandot    75980  2399942   
1354    OH       39       175    Wyandot    79044  2397094   
1355    OH       39       175    Wyandot    84574  2400159   

                 PLACENAME                     TYPE CLASSFP FUNCSTAT  ...  \
0          Bentonville CDP  CENSUS DESIGNATED PLACE      U1        S  ...   
1          Cherry Fork CDP  CENSUS DESIGNATED PLACE      U1        S  ...   
2       Manchester villa

In [78]:
# df = pd.read_excel("median income bby city.xlsx", names = ['City', 'Income'])
# total_df = total_df.merge(df, on = 'City', how = 'inner')
# total_df

In [134]:
df = pd.read_csv("hospital.csv", sep = '\t')
# print(df)

# print(df[df['City'] == 'Columbus'])

total_df = tot_df.merge(df, left_on = 'Key', right_on = 'City', how = 'left')
total_df[total_df['Key'] == 'Columbus']

,STATE,STATEFP,COUNTYFP,COUNTYNAME,PLACEFP,PLACENS,PLACENAME,TYPE,CLASSFP,FUNCSTAT,...,medinc_23p4,medinc_23p5,medinc_23p6,medinc_23p7,medinc_23p8,City_x,crime_rate,Unnamed: 0_y,City_y,Number of Hospitals
319,OH,39,41,Delaware,18000,1086101,Columbus city,INCORPORATED PLACE,C6,A,...,99200,107200,115100,123100,131000,NaN,NaN,25.0,Columbus,7.0
351,OH,39,45,Fairfield,18000,1086101,Columbus city,INCORPORATED PLACE,C6,A,...,99200,107200,115100,123100,131000,NaN,NaN,25.0,Columbus,7.0
379,OH,39,49,Franklin,18000,1086101,Columbus city,INCORPORATED PLACE,C6,A,...,99200,107200,115100,123100,131000,NaN,NaN,25.0,Columbus,7.0


In [139]:
final = total_df.groupby(['Key', 'COUNTYNAME']).mean()
final = final.reset_index()
print(final.columns)
FIN = final.drop(columns=['STATEFP', 'COUNTYFP', 'PLACEFP', 'PLACENS', 'Unnamed: 0_x', 'Unnamed: 0_y','FIPS State Code', 'FIPS County Code', 'medinc_23p5', 'medinc_23p6', 'medinc_23p7', 'medinc_23p8'])
FIN

Index(['Key', 'COUNTYNAME', 'STATEFP', 'COUNTYFP', 'PLACEFP', 'PLACENS',
       'DIFFSklPI', 'DIFFBuildPI', 'Unnamed: 0_x', 'FIPS State Code',
       'FIPS County Code', 'medinc_23p1', 'medinc_23p2', 'medinc_23p3',
       'medinc_23p4', 'medinc_23p5', 'medinc_23p6', 'medinc_23p7',
       'medinc_23p8', 'crime_rate', 'Unnamed: 0_y', 'Number of Hospitals'],
      dtype='object')


,Key,COUNTYNAME,DIFFSklPI,DIFFBuildPI,medinc_23p1,medinc_23p2,medinc_23p3,medinc_23p4,crime_rate,Number of Hospitals
0,Aberdeen,Brown,15.525,28.562,54900,62800,70600,78400,NaN,NaN
1,Ada,Hardin,12.736,18.519,54900,62800,70600,78400,0.980214,NaN
2,Adamsville,Muskingum,48.627,59.376,54900,62800,70600,78400,NaN,NaN
3,Addyston,Hamilton,76.719,84.015,70800,80900,91000,101100,NaN,NaN
4,Adelphi,Ross,17.021,24.296,54900,62800,70600,78400,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1351,Youngstown,Trumbull,50.854,58.577,54900,62800,70600,78400,NaN,1.0
1352,Zaleski,Vinton,0.000,29.410,54900,62800,70600,78400,NaN,NaN
1353,Zanesfield,Logan,9.722,16.627,59100,67500,75900,84300,NaN,NaN
1354,Zanesville,Muskingum,48.627,59.376,54900,62800,70600,78400,9.165154,1.0


In [140]:
FIN.to_csv('data/final.csv', sep = '\t')

In [142]:
print(final[final['Key'] == 'Bexley'])
print(final[final['Key'] == 'Big Walnut'])
print(final[final['Key'] == 'Blacklick'])
print(final[final['Key'] == 'Columbus'])
print(final[final['Key'] == 'Dublin'])
print(final[final['Key'] == 'Gahanna'])
print(final[final['Key'] == 'Grandview'])
print(final[final['Key'] == 'Hilliard'])
print(final[final['Key'] == 'Licking Heights'])
print(final[final['Key'] == 'Lewis Center'])
print(final[final['Key'] == 'New Albany'])
print(final[final['Key'] == 'Olentangy Local'])
print(final[final['Key'] == 'Reynoldsburg'])
print(final[final['Key'] == 'Sunbury'])
print(final[final['Key'] == 'Upper Arlington'])
print(final[final['Key'] == 'Westerville'])
print(final[final['Key'] == 'Worthington'])

        Key COUNTYNAME  STATEFP  COUNTYFP  PLACEFP  PLACENS  DIFFSklPI  \
103  Bexley   Franklin       39        49     6278  1086097     69.463   

     DIFFBuildPI  Unnamed: 0_x  FIPS State Code  ...  medinc_23p2  \
103         82.5            24               39  ...        79400   

     medinc_23p3  medinc_23p4  medinc_23p5  medinc_23p6  medinc_23p7  \
103        89300        99200       107200       115100       123100   

     medinc_23p8  crime_rate  Unnamed: 0_y  Number of Hospitals  
103       131000    4.471195           NaN                  NaN  

[1 rows x 22 columns]
Empty DataFrame
Columns: [Key, COUNTYNAME, STATEFP, COUNTYFP, PLACEFP, PLACENS, DIFFSklPI, DIFFBuildPI, Unnamed: 0_x, FIPS State Code, FIPS County Code, medinc_23p1, medinc_23p2, medinc_23p3, medinc_23p4, medinc_23p5, medinc_23p6, medinc_23p7, medinc_23p8, crime_rate, Unnamed: 0_y, Number of Hospitals]
Index: []

[0 rows x 22 columns]
Empty DataFrame
Columns: [Key, COUNTYNAME, STATEFP, COUNTYFP, PLACEFP, PLAC